In [ ]:
!pip install -r requirements.txt --ignore-installed --quiet

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf

from mlxtend.frequent_patterns import apriori, association_rules
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Concatenate, Dense, Embedding, Flatten, Input
from tensorflow.keras.models import Model


print(f"Versão TensorFlow: `{tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs disponíveis: `{gpus if gpus else 'Não detectado nenhum GPU'}`")

In [ ]:
df = pd.read_excel(
    "Pedidos.xlsx",
    parse_dates=["DataFechamento"]  # Converte diretamente essa coluna para datetime
)
df.head()

In [ ]:
# pega todas as colunas numéricas
num_cols = df.select_dtypes(include="number").columns

# remove qtde_vendida da lista
num_cols = num_cols.drop("qtde_vendida")

# arredonda apenas as numéricas restantes
df[num_cols] = df[num_cols].round(2)

df.head()


In [ ]:
# Dimensão
print("Formato:", df.shape)

# Tipos de dados
print("\nTipos:")
print(df.dtypes)

# Estatísticas numéricas
print("\nEstatísticas:")
print(df.describe())

# Ver valores nulos
print("\nNulos:")
print(df.isnull().sum()) 


In [ ]:
# Ver linhas com valores negativos
linhas_removidas = (df["qtde_vendida"] < 0).sum()
print("Linhas a remover:", linhas_removidas)

df = df[df["qtde_vendida"] >= 0].copy()  

df = df[df["DataFechamento"].dt.year < 2025].copy()


In [ ]:
df = df.drop_duplicates()

In [ ]:
df["valor_calculado"] = df["qtde_vendida"] * df["valor_unitario"] + df["acrescimos"] - df["descontos"]

# Diferença entre o calculado e o informado
df["diferenca"] = (df["valor_total"] - df["valor_calculado"]).round(2)

# Conferir registros inconsistentes
inconsistentes = df[df["diferenca"] != 0]


In [ ]:
# Dimensão
print("Formato:", df.shape)

In [ ]:
# Z-score da coluna 'valor_pedido'
z_scores = stats.zscore(df['valor_pedido'])
df_sem_outliers = df[(z_scores > -3) & (z_scores < 3)].copy()

In [ ]:
# --- Agregações solicitadas ---
total_pedidos = df_sem_outliers['numero_pedido'].nunique()
valor_medio_pedido = df_sem_outliers['valor_pedido'].mean()
maior_valor_pedido = df_sem_outliers['valor_pedido'].max()
menor_valor_pedido = df_sem_outliers['valor_pedido'].min()
total_itens_vendidos = df_sem_outliers['qtde_vendida'].sum()

# Vendas por categoria
vendas_por_categoria = df_sem_outliers.groupby('descricao_categoria')['valor_total'].sum().reset_index()
vendas_por_categoria = vendas_por_categoria.rename(columns={'valor_total': 'vendas_categoria'})

# Vendas ano a ano
df_sem_outliers['ano'] = df_sem_outliers['DataFechamento'].dt.year
vendas_ano_a_ano = df_sem_outliers.groupby('ano')['valor_total'].sum().reset_index()
vendas_ano_a_ano = vendas_ano_a_ano.rename(columns={'valor_total': 'vendas_ano'})

# Criando um DataFrame resumo geral
resumo_geral = pd.DataFrame({
    'total_pedidos': [total_pedidos],
    'valor_medio_pedido': [valor_medio_pedido],
    'maior_valor_pedido': [maior_valor_pedido],
    'menor_valor_pedido': [menor_valor_pedido],
    'total_itens_vendidos': [total_itens_vendidos]
})

# Mostrando os resultados
print("📊 Resumo Geral:")
print(resumo_geral)
print("\n🏷️ Vendas por categoria:")
print(vendas_por_categoria)
print("\n📅 Vendas ano a ano:")
print(vendas_ano_a_ano)

In [ ]:
print("\n📅 Vendas ano a ano:")
print(vendas_ano_a_ano)

In [ ]:
total_itens_diferentes = df['codigo_produto'].nunique()
print("🛒 Total de itens diferentes:", total_itens_diferentes)

In [ ]:
# Histograma de valores unitários
df_sem_outliers["valor_unitario"].hist(bins=30)
plt.title("Distribuição do Valor Unitário")
plt.xlabel("Valor Unitário")
plt.ylabel("Frequência")
plt.show()

# Top 10 produtos mais vendidos
df_sem_outliers["descricao_produto"].value_counts().head(10).plot(kind="bar")
plt.title("Top 10 Produtos Mais Vendidos")
plt.show()


In [ ]:
# Receita total por categoria
df_sem_outliers.groupby("descricao_categoria")["valor_total"].sum().sort_values(ascending=False).plot(kind="bar")
plt.title("Receita Total por Categoria")
plt.ylabel("Receita")
plt.show()

# Receita ao longo do tempo
df_sem_outliers.groupby(df_sem_outliers["DataFechamento"].dt.date)["valor_total"].sum().plot()
plt.title("Receita ao Longo do Tempo")
plt.ylabel("Receita")
plt.show()

In [ ]:
# ================================
# Receita total por subcategoria
# ================================
df.groupby("descricao_subcategoria")["valor_total"].sum().sort_values(ascending=False).plot(kind="bar", figsize=(12,6))
plt.title("Receita Total por Subcategoria")
plt.ylabel("Receita")
plt.xlabel("Subcategoria")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Total vendido por categoria
vendas_categoria = df.groupby("descricao_categoria")["valor_total"].sum().sort_values(ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(
    x=vendas_categoria.index,
    y=vendas_categoria.values,
    hue=vendas_categoria.index,
    palette="viridis",
    legend=False
)
plt.xticks(rotation=45)
plt.ylabel("Faturamento Total")
plt.title("Faturamento por Categoria")
plt.show()

In [ ]:
# Top 10 produtos por quantidade vendida
top_produtos = df.groupby("descricao_produto")["qtde_vendida"].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.barplot(
    x=top_produtos.index,
    y=top_produtos.values,
    hue=top_produtos.index,
    palette="magma",
    legend=False
)
plt.xticks(rotation=45)
plt.ylabel("Quantidade Vendida")
plt.title("Top 10 Produtos Mais Vendidos")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df["valor_unitario"], bins=50, kde=True)
plt.xlabel("Valor Unitário")
plt.title("Distribuição do Valor Unitário dos Produtos")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df[["qtde_vendida","valor_unitario","acrescimos","descontos","valor_total","valor_pedido"]].corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre Variáveis Numéricas")
plt.show()

In [ ]:
# 1️⃣ Criar matriz binária pedido x produto
matriz_binaria = df.pivot_table(
    index='numero_pedido',
    columns='descricao_produto',
    values='qtde_vendida',
    fill_value=0
).map(lambda x: 1 if x > 0 else 0)

# 2️⃣ Filtrar os 100 produtos mais vendidos
top_100_produtos = df.groupby('descricao_produto')['qtde_vendida'].sum().sort_values(ascending=False).head(100).index
matriz_top100 = matriz_binaria[top_100_produtos]

# 3️⃣ Aplicar Apriori para conjuntos maiores
frequent_itemsets = apriori(
    matriz_top100,
    min_support=0.005,   # itemset deve aparecer em pelo menos 1% dos pedidos
    use_colnames=True,
    max_len=5           # pegando conjuntos de até 5 produtos
)

# 4️⃣ Gerar regras de associação mais completas
regras = association_rules(
    frequent_itemsets,
    metric="lift",
    min_threshold=1.0
).sort_values(by=['confidence', 'lift'], ascending=False)

# 5️⃣ Criar uma matriz de relações entre todos os produtos
# Inicializa DataFrame vazio
produtos = matriz_top100.columns.tolist()
matriz_relacoes = pd.DataFrame(0, index=produtos, columns=produtos, dtype=float)

# Preenche a matriz com a confiança das regras
for _, row in regras.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])
    for a in antecedents:
        for c in consequents:
            if a != c:
                matriz_relacoes.loc[a, c] = max(matriz_relacoes.loc[a, c], row['confidence'])

In [ ]:
# 6️⃣ Função para pegar os top N produtos relacionados
def produtos_relacionados_avancado_multi(produtos_input, n=5):
    """
    Retorna os n produtos mais relacionados a um ou mais produtos.
    
    produtos_input: str ou lista de str
    n: número de produtos relacionados a retornar
    """
    if isinstance(produtos_input, str):
        produtos_input = [produtos_input]
    
    # Verifica se os produtos existem na matriz
    produtos_validos = [p for p in produtos_input if p in matriz_relacoes.index]
    if not produtos_validos:
        return f"Nenhum dos produtos informados foi encontrado na matriz."

    # Soma as relações dos produtos informados
    soma_relacoes = matriz_relacoes.loc[produtos_validos].sum(axis=0)
    
    # Remove os próprios produtos da lista
    soma_relacoes = soma_relacoes.drop(produtos_validos, errors='ignore')
    
    # Filtra apenas produtos com relação > 0 e ordena
    soma_relacoes = soma_relacoes[soma_relacoes > 0].sort_values(ascending=False)
    
    return soma_relacoes.head(n)

# Realizando "Predição" com Apriori 
print(produtos_relacionados_avancado_multi(["DOGAO 1", "SALADA"]))

In [ ]:
# Supondo que seu DataFrame original se chame 'df'
# e já esteja carregado com os dados que você me mostrou
# 1. Criar um DataFrame de interações
# Filtrar apenas as vendas (qtde_vendida > 0)
interacoes = df[df['qtde_vendida'] > 0].copy()

# 2. Codificar os IDs de pedidos e produtos
# A rede neural precisa de números inteiros, não de strings.
pedido_encoder = LabelEncoder()
interacoes['pedido_id'] = pedido_encoder.fit_transform(interacoes['numero_pedido'])

produto_encoder = LabelEncoder()
interacoes['produto_id'] = produto_encoder.fit_transform(interacoes['descricao_produto'])

# Mapear IDs de volta para nomes para referência futura
produtos_map = {idx: produto for idx, produto in enumerate(produto_encoder.classes_)}
pedidos_map = {idx: pedido for idx, pedido in enumerate(pedido_encoder.classes_)}

# Número total de pedidos e produtos únicos
n_pedidos = len(interacoes['pedido_id'].unique())
n_produtos = len(interacoes['produto_id'].unique())

print(f"Total de pedidos únicos: {n_pedidos}")
print(f"Total de produtos únicos: {n_produtos}")

# 3. Separar os dados em treino e teste
X = interacoes[['pedido_id', 'produto_id']]
# A variável 'y' representa a interação.
# Como já filtramos para qtde_vendida > 0, todos os valores são 1.
# Não precisamos de uma variável 'y' explícita neste modelo, pois
# a interação é o próprio par (pedido, produto)
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [ ]:
# Tamanho do embedding (hiperparâmetro)
embedding_size = 50

# Inputs
pedido_input = Input(shape=(1,))
produto_input = Input(shape=(1,))

# Embeddings
pedido_embedding = Embedding(input_dim=n_pedidos, output_dim=embedding_size, name='pedido_embedding')(pedido_input)
produto_embedding = Embedding(input_dim=n_produtos, output_dim=embedding_size, name='produto_embedding')(produto_input)

# Flatten
pedido_vec = Flatten()(pedido_embedding)
produto_vec = Flatten()(produto_embedding)

# Concatenar embeddings
x = Concatenate()([pedido_vec, produto_vec])

# Camadas densas
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Probabilidade de interação

# Modelo
model = Model(inputs=[pedido_input, produto_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
# Criar pares negativos
n_positivos = len(X_train)
neg_pedidos = np.random.choice(X_train['pedido_id'], size=n_positivos)
neg_produtos = np.random.choice(X_train['produto_id'], size=n_positivos)

# Combinar positivos e negativos
X_train_neg = pd.DataFrame({
    'pedido_id': np.concatenate([X_train['pedido_id'], neg_pedidos]),
    'produto_id': np.concatenate([X_train['produto_id'], neg_produtos])
})
y_train = np.concatenate([np.ones(n_positivos), np.zeros(n_positivos)])

# Mesmo para teste
n_test = len(X_test)
neg_pedidos_test = np.random.choice(X_test['pedido_id'], size=n_test)
neg_produtos_test = np.random.choice(X_test['produto_id'], size=n_test)
X_test_neg = pd.DataFrame({
    'pedido_id': np.concatenate([X_test['pedido_id'], neg_pedidos_test]),
    'produto_id': np.concatenate([X_test['produto_id'], neg_produtos_test])
})
y_test = np.concatenate([np.ones(n_test), np.zeros(n_test)])


In [ ]:
history = model.fit(
    [X_train_neg['pedido_id'], X_train_neg['produto_id']],
    y_train,
    validation_data=([X_test_neg['pedido_id'], X_test_neg['produto_id']], y_test),
    epochs=10,
    batch_size=256
)